<a href="https://colab.research.google.com/github/JavZon/Machine_Learning/blob/main/Cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

Datasetni chaqiramiz

In [2]:
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)
X_num=df.drop(['median_house_value','ocean_proximity'],axis=1)

**Transformerni** chaqiramiz

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
      pass
    def fit(self, X, y=None):
      return self
    def transform(self, X):
      rooms_per_household=X[:, rooms_ix] / X[:, households_ix]
      population_per_household=X[:, population_ix] / X[:, households_ix]
      return np.c_[X, rooms_per_household, population_per_household]

**Numerical** pipeline

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

numeric_pipeline=Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('custom_obj_adder', CustomTransformer()),
    ('std_scaler', StandardScaler())
])

**Full pipeline**

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_cols=list(X_num.columns)
string_cols=['ocean_proximity']

full_pipeline=ColumnTransformer([
    ('numeric', numeric_pipeline, num_cols),
    ('string_cat',OneHotEncoder(handle_unknown='ignore'), string_cols)
])

**Cross Validation** usuli

Bu usul datasetni train va test setlarga ajratmaydi, balki biz necha qismga ajrat desak o'shancha qismga ajratadi va 1 qismini test uchun olib qo'yadi va qolganlarini train uchun ishlatadi. Masalan k=5 (5 qismga ajrat desak) butun datasetni qismga bo'ladi va modelning aniqligini 5 marta hisoblaydi. Bunda 1-martasida 1-qism test uchun olib qo'yiladi 2,3,4,5-qismlar train qilinadi. 2-martasida 2-qism test, 1,3,4,5 train bo'ladi. 3-martada 3-qism test, 1,2,4,5 train, 4-martada 4-qism test, 1,2,3,5 train va 5-martada 5-qism test 1,2,3,4 train bo'ladi. Har biri uchun alohida-alohida rmse hisoblaydi. Biz shu RMSE larni qo'shib 5ga bo'lsak butun dataset uchun yakuniy RMSE kelib chiqadi

In [6]:
from sklearn.model_selection import cross_val_score

Datasetni tayyorlab olamiz

In [7]:
X = df.drop("median_house_value", axis=1) # Labelni olib tashlab X featuresning o'zini qoldiramiz
y = df["median_house_value"].copy() # Labelni saqlab olamiz

X_num=df.drop(['median_house_value','ocean_proximity'], axis=1) # sonli columnlar ishlashi uchun faqat labelni va category columni drop qilamiz

X_prepared = full_pipeline.fit_transform(X)

**LinearRegression** ni chaqirib model yaratamiz

In [8]:
from sklearn.linear_model import LinearRegression
LR_model=LinearRegression()

**RMSE** using LR_model

In [17]:
scores = cross_val_score(LR_model, X_prepared, y, scoring="neg_root_mean_squared_error", cv=5)
print(-scores)
print("RMSE =",-scores.mean())

[73401.50237805 75581.31110081 75683.02945871 77187.27760314
 66370.08542016]
RMSE = 73644.64119217482


**RandomForest** ni chaqirib model yaratamiz

In [10]:
from sklearn.ensemble import RandomForestRegressor
RF_model=RandomForestRegressor()

**RMSE** using RF_model

In [18]:
scores=cross_val_score(RF_model, X_prepared, y, scoring="neg_root_mean_squared_error", cv=5)
print(-scores)
print("RMSE =",-scores.mean())

[76726.61902321 63789.37852752 61633.31824663 80609.56949339
 62468.35657213]
RMSE = 69045.44837257519


**Decision Tree** Algoritmi

In [19]:
from sklearn.tree import DecisionTreeRegressor
Tree_model = DecisionTreeRegressor()

RMSE using Tree_model

In [20]:
scores=cross_val_score(Tree_model, X_prepared, y, scoring="neg_root_mean_squared_error", cv=5)
print(-scores)
print("RMSE =",-scores.mean())

[ 99767.34295428  82920.29099136  82709.4555248  108394.40629504
  82416.72745469]
RMSE = 91241.64464403316
